<a href="https://colab.research.google.com/github/ShadowRaccoon/PrograConcurrente2023C1/blob/main/TP1_threads_Cpp_M1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **C++**

In [3]:
%%writefile matrix_mult.cpp
#include <iostream>
#include <vector>
#include <string>
#include <random>
#include <utility>
#include <thread>
#include <stdlib.h>

#define MIN_MATRIX_SIZE 5
#define MAX_MATRIX_SIZE 20
#define MIN_MATRIX_CELL_VALUE -32
#define MAX_MATRIX_CELL_VALUE 32

typedef std::vector<std::vector<int>> Matrix;

int validateArgs(int& argc, char* argv[]);
Matrix matrixGenerator(int size, bool populate = true);
Matrix matrixGenerator(int row_size, int col_size, bool populate = true);
Matrix secuentialMult(Matrix&, Matrix&);
Matrix concurrentMult(Matrix&, Matrix&);
void printMatrix(std::string alias, Matrix&);

int main(int argc, char* argv[])
{
    int size = validateArgs(argc, argv);

    Matrix matrix_a  {matrixGenerator(size)};
    Matrix matrix_b  {matrixGenerator(size)};
    Matrix matrix_cs {secuentialMult(matrix_a, matrix_b)};
    Matrix matrix_ch {concurrentMult(matrix_a, matrix_b)};

    printMatrix("A", matrix_a);
    printMatrix("B", matrix_b);
    printMatrix("CS", matrix_cs);
    printMatrix("CH", matrix_ch);
    std::cout << "Las matrices CS y CH son iguales? "
        // El operador == en vectores compara todos los elementos del vector
        << (matrix_cs == matrix_ch ? "Si" : "No")
        << std::endl;

    return EXIT_SUCCESS;
}

// Valida los argumentos y devuelve el tamaño de la matrices a crear
int validateArgs(int& argc, char* argv[])
{
    if (argc != 2)
    {
        std::cout << "Cantidad de argumentos incorrectos" << std::endl;
        exit(EXIT_FAILURE);
    }

    int size = atoi(argv[1]);
    if (size < MIN_MATRIX_SIZE || size > MAX_MATRIX_SIZE)
    {
        std::cout << "El tamaño de la matriz debe estar entre "
            << MIN_MATRIX_SIZE << " y " << MAX_MATRIX_SIZE << std::endl;

        exit(EXIT_FAILURE);
    }

    return size;
}

// Genera una matriz de igual tamaño de filas y columnas
Matrix matrixGenerator(int size, bool populate)
{
    // "Movemos" al objeto en vez de realizar una copia
    return std::move(matrixGenerator(size, size, populate));
}

// Genera una matriz y las llena de datos de ser necesario
Matrix matrixGenerator(int row_size, int col_size, bool populate)
{
    Matrix aux(row_size, std::vector<int>(col_size));

    if (populate)
    {
        std::random_device rd; // Obtenemos un número random del hardware
        std::mt19937 gen(rd()); // Generamos la semilla (seed)
        std::uniform_int_distribution<> distr(MIN_MATRIX_CELL_VALUE, MAX_MATRIX_CELL_VALUE); // Definimos el rango

        for (int i=0; i < row_size; ++i)
            for (int j=0; j < col_size; ++j)
                aux[i][j] = distr(gen);
    }

    // "Movemos" al objeto en vez de realizar una copia
    return std::move(aux);
}

// Realiza la multiplicación entre 2 matrices y devuelve una nueva de manera secuencial
Matrix secuentialMult(Matrix& a, Matrix& b)
{
    Matrix aux = matrixGenerator(a.size(), false);

    for (int i=0; i < a.size(); ++i)
        for (int j=0; j < b[0].size(); ++j)
            for (int k=0; k < a.size(); ++k)
                aux[i][j] += a[i][k] * b[k][j];    

    // "Movemos" al objeto en vez de realizar una copia
    return std::move(aux);
}

// Realiza la multiplicación entre 2 matrices y devuelve una nueva de manera concurrente
Matrix concurrentMult(Matrix& a, Matrix& b)
{
    Matrix aux = matrixGenerator(a.size(), false);
    std::vector<std::thread> workers;

    for (int i; i < a.size(); ++i)
    {
        workers.push_back(std::thread(
                // Función anónima o lambda que realiza la asignación y multiplicación
                [&aux, &a, &b, i]()
                {
                    for (int j=0; j < b[0].size(); ++j)
                        for (int k=0; k < a.size(); ++k)
                            aux[i][j] += a[i][k] * b[k][j];
                }
            )
        );
    }
    
    for(auto& worker : workers)
    {
        worker.join();
    }

    // "Movemos" al objeto en vez de realizar una copia
    return std::move(aux);
}

// Imprime la matriz con su alias
void printMatrix(std::string alias, Matrix& m)
{
    std::cout << "Matrix " << alias << ":" << std::endl;
    
    for(const auto& row : m)
    {
        for(const auto& col : row)
        {
            std::cout << "\t" << col << "\t";
        }
        std::cout << std::endl;
    }
}


Overwriting matrix_mult.cpp


In [4]:
!g++ matrix_mult.cpp -pthread -std=c++14 -o matrix_mult.out

In [5]:
!./matrix_mult.out 1

El tamaño de la matriz debe estar entre 5 y 20


In [6]:
!./matrix_mult.out 5

Matrix A:
	31		-25		8		-1		-28	
	-2		6		0		19		-24	
	-20		2		25		6		-18	
	31		-10		16		-14		17	
	27		12		-5		8		24	
Matrix B:
	27		-26		16		12		-9	
	-9		-6		-7		9		21	
	10		-2		-31		-13		-1	
	15		5		28		0		27	
	0		-10		-10		-2		19	
Matrix CS:
	1127		-397		675		99		-1371	
	177		351		698		78		201	
	-218		668		-761		-511		17	
	877		-1018		-492		40		-560	
	691		-964		487		449		686	
Matrix CH:
	1127		-397		675		99		-1371	
	177		351		698		78		201	
	-218		668		-761		-511		17	
	877		-1018		-492		40		-560	
	691		-964		487		449		686	
Las matrices CS y CH son iguales? Si
